In [5]:
import pandas as pd

In [6]:
df = pd.read_excel('./my_data/train/Real-Time-Voice-Cloning.xlsx')

### Add Length of title and description

In [7]:
df.head(1)

,fullname,number,html_url,title,description,labels
0,iperov/DeepFaceLab,1,https://github.com/iperov/DeepFaceLab/issues/1,"`Error: Sorry, this model works only on 2GB+ G...",## Expected behavior_x000D_\nStart training._x...,other


In [8]:
def _strlen(sentence):
    import re
    zhmodel = re.compile(u'[\u4e00-\u9fa5]')  #检查中文
    contents = str(sentence)
    match = zhmodel.search(contents)
    if match:
        return -1
    else:
        return len(str(sentence).strip().split(' '))

title_lens, body_lens = df['title'].apply(_strlen), df['description'].apply(_strlen)

In [9]:
cols = df.columns
df.insert(loc=df.columns.get_loc("title") + 1, column='title_lens', value=title_lens)
df.insert(loc=df.columns.get_loc("description") + 1, column='description_lens', value=body_lens)

In [10]:
df.head(1)

,fullname,number,html_url,title,title_lens,description,description_lens,labels
0,iperov/DeepFaceLab,1,https://github.com/iperov/DeepFaceLab/issues/1,"`Error: Sorry, this model works only on 2GB+ G...",15,## Expected behavior_x000D_\nStart training._x...,375,other


### Remove nan Label and short title with no description

In [11]:
df['labels'].unique()

array(['other', 'Low efficiency and Effectiveness', 'deployment', 'Error',
       'tensor&inputs'], dtype=object)

In [12]:
contain_na = df[df['labels'].isnull()]
contain_na.head(3)

,fullname,number,html_url,title,title_lens,description,description_lens,labels


In [13]:
len_before = len(df)
df.dropna(subset = ['labels'], inplace=True)
len_after = len(df)
print(f'before:{len_before}, after:{len_after}')

before:716, after:716


In [14]:
contain_chinese_characters = df[((df.title_lens == -1) & (df.description_lens == -1))]
df = df.drop(contain_chinese_characters.index)

In [15]:
body_nan = df[((df.title_lens == 1) & (df.description.isnull()))]
df = df.drop(body_nan.index)

In [16]:
df = df.sort_values(by=['title_lens', 'description_lens'])

# save the new file

In [17]:
name = '_'.join(df['fullname'][0].split('/'))
name

'iperov_DeepFaceLab'

In [18]:
if not os.path.exists(os.path.join('./my_data', name)):
    os.mkdir(os.path.join('./my_data', name))

In [19]:
df.to_excel(f'{name}.xlsx')

In [20]:
import json
result = df[['title', 'description', 'labels']].to_json(orient="records")
parsed = json.loads(result)

with open(f'{name}.txt', 'w') as f:
    json.dump(parsed, f)

In [21]:
data = []
with open('deepfakes_faceswap.txt', 'r', encoding='utf-8') as f:
    data = json.load(f)

for obj in data:
    print(obj)
    break

{'title': 'setUpfailed', 'description': '![default](https://user-images.githubusercontent.com/28338863/53715027-7e309480-3e8b-11e9-8d5b-aebb44e63f9e.png)\r\n\r\ndear editor,what does these mean?', 'labels': 'other'}


## Combine the code

In [39]:
import os
import glob
import json
import pandas as pd

In [40]:
def _strlen(sentence):
    import re
    zhmodel = re.compile(u'[\u4e00-\u9fa5]')  #检查中文
    contents = str(sentence)
    match = zhmodel.search(contents)
    if match:
        return -1
    else:
        return len(str(sentence).strip().split(' '))

def concatenate_df_and_save(dfs):
    df = pd.concat(dfs)
    dir_name = os.path.join('./my_data/train', 'concat')
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    
    out_path = os.path.join(dir_name, 'concat.txt')
    result = df[['title', 'description', 'labels']].to_json(orient="records")
    parsed = json.loads(result)
    with open(out_path, 'w') as f:
        json.dump(parsed, f)

def save_df(df, filename):
    dir_name = os.path.join('./my_data/train', filename)
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    
    out_path = os.path.join(dir_name, filename + '.txt')
    result = df[['title', 'description', 'labels']].to_json(orient="records")
    parsed = json.loads(result)
    with open(out_path, 'w') as f:
        json.dump(parsed, f)
    
def preprocess(file):
    df = pd.read_excel(file)
    # add title and description length
    title_lens, description_lens = df['title'].apply(_strlen), df['description'].apply(_strlen)

    df.insert(loc=df.columns.get_loc("title") + 1, column='title_lens', value=title_lens)
    df.insert(loc=df.columns.get_loc("description") + 1, column='description_lens', value=description_lens)
    
    name = '_'.join(df['fullname'][0].split('/'))
    print(f'before drop, the length of {name} is:{len(df)}')
    
    # drop the nan
    df.dropna(subset = ['title', 'description', 'labels'], inplace=True)
    
    # drop the chinese issues
    contain_chinese_characters = df[((df.title_lens == -1) & (df.description_lens == -1))]
    df.drop(contain_chinese_characters.index, inplace=True)
    
    print(f'after drop, the length of {name} is:{len(df)}')

    # whether or not to convert erery xlxs file to json file
    
    # name = '_'.join(df['fullname'][0].split('/'))
    # dir_name = os.path.join('./my_data', name)
    # if not os.path.exists(dir_name):
    #     os.mkdir(dir_name)
    
    # out_path = os.path.join(dir_name, f'{name}.txt')
    # result = df[['title', 'description', 'labels']].to_json(orient="records")
    # parsed = json.loads(result)

    # with open(out_path, 'w') as f:
    #     json.dump(parsed, f)
    
    return df
    

### Concat train file

In [37]:
# dfs = []
# dfs.append(preprocess('my_data/train/pytorch-CycleGAN-and-pix2pix_TRAIN_Aug.xlsx'))
# dfs.append(preprocess('my_data/train/Real-Time-Voice-Cloning_TRAIN_Aug.xlsx'))
# dfs.append(preprocess('my_data/train/EasyOCR_TRAIN_Aug.xlsx'))
# dfs.append(preprocess('my_data/train/recommenders1_TRAIN_Aug.xlsx')) 
# # dfs.append(preprocess('my_data/train/streamlit1_TRAIN_Aug.xlsx'))
# concatenate_df_and_save(dfs)

In [41]:
#save_df(preprocess('my_data/train/TTS3_TRAIN_Aug.xlsx'), 'TTS3_TRAIN_Aug')
#save_df(preprocess('my_data/train/Real-Time-Voice-Cloning_TRAIN_Aug.xlsx'), 'Real-Time-Voice-Cloning_TRAIN_Aug')
#save_df(preprocess('my_data/train/EasyOCR_TRAIN_Aug.xlsx'), 'EasyOCR_TRAIN_Aug')
#save_df(preprocess('my_data/train/recommenders3_TRAIN_Aug.xlsx'), 'recommenders3_TRAIN_Aug') 
#save_df(preprocess('my_data/train/streamlit1_TRAIN_Aug.xlsx'), 'streamlit1_TRAIN_Aug')
#save_df(preprocess('my_data/train/faceswap_TRAIN_Aug.xlsx'), 'faceswap_TRAIN_Aug')
#save_df(preprocess('my_data/train/deepfacelab1_TRAIN_Aug.xlsx'), 'deepfacelab1_TRAIN_Aug')
#save_df(preprocess('my_data/train/openpose0_TRAIN_Aug.xlsx'), 'openpose0_TRAIN_Aug')
save_df(preprocess('my_data/train/contact2_TRAIN_Aug.xlsx'), 'contact2_TRAIN_Aug')

before drop, the length of deezer_spleeter is:1968
after drop, the length of deezer_spleeter is:1920


In [9]:
files = glob.glob('./my_data/train/*.xlsx')
for f in files:
    print(f)

dfs = []
for f in files:
    dfs.append(preprocess(f))
concatenate_df_and_save(dfs)    

./my_data/train\deepfacelab.xlsx
./my_data/train\EasyOCR.xlsx
./my_data/train\EasyOCR_TRAIN_Aug.xlsx
./my_data/train\faceswap.xlsx
./my_data/train\faceswap_TRAIN_Aug.xlsx
./my_data/train\pytorch-CycleGAN-and-pix2pix.xlsx
./my_data/train\pytorch-CycleGAN-and-pix2pix_TRAIN_Aug.xlsx
./my_data/train\Real-Time-Voice-Cloning.xlsx
./my_data/train\Real-Time-Voice-Cloning_TRAIN_Aug.xlsx
./my_data/train\recommenders.xlsx
./my_data/train\recommenders1.xlsx
./my_data/train\recommenders1_TRAIN_Aug.xlsx
./my_data/train\recommenders_TRAIN_Aug.xlsx
./my_data/train\spleeter.xlsx
./my_data/train\streamlit.xlsx
./my_data/train\streamlit1.xlsx
./my_data/train\streamlit1_TRAIN_Aug.xlsx
./my_data/train\TTS.xlsx
before drop, the length of iperov_DeepFaceLab is:716
after drop, the length of iperov_DeepFaceLab is:701
before drop, the length of JaidedAI_EasyOCR is:544
after drop, the length of JaidedAI_EasyOCR is:516
before drop, the length of JaidedAI_EasyOCR is:615
after drop, the length of JaidedAI_EasyOCR i

## process one file

In [4]:
test_file = './my_data/test/TTS.xlsx'
def save_test_to_txt(file):
    df = preprocess(file)
    file_name = file.split('/')[-1].split('.')[0]
    dir_name = os.path.join('./my_data/test', file_name)
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    
    out_path = os.path.join(dir_name, f'{file_name}.txt')
    result = df[['title', 'description', 'labels']].to_json(orient="records")
    parsed = json.loads(result)
    with open(out_path, 'w') as f:
        json.dump(parsed, f)

save_test_to_txt(test_file)

before drop, the length of mozilla_TTS is:506
after drop, the length of mozilla_TTS is:478
